
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Securing Data in Unity Catalog

In this demo you will learn how to hide sensitive data using 3 different approaches:
* Views 
* Dynamic Views
* Row Filter and Column Masks on Tables (introduced in 2024)

Further, you will also learn data governance features of Unity Catalog
* Introduction to Catalog Explorer 
* Enable data access to users using inherited and explicit privileges 
* Tagging + AI generated Documentation
* Use Lineage and Insight features in Unity Catalog to understand data flow and access patterns.


## REQUIRED - SELECT SERVERLESS

Before executing cells in this notebook, please select Serverless cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the Serverless cluster:

- Navigate to the top-right of this notebook and click the drop-down menu to select Serverless. By default, the notebook will use **Serverless**.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your unique catalog name and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG your-catalog;
USE SCHEMA your-catalog.pii_data;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-connect 17.2.4 requires databricks-sdk>=0.46.0, but you have databricks-sdk 0.36.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


NOTE: Table created: labuser13495410_1768540662.pii_data.customers_bronze
NOTE: Table created: labuser13495410_1768540662.pii_data.customers_silver


Your Course Catalog Name:,
Your Course Schema:,


Run the code below to view your current default catalog and schema. Confirm that they have the same name as the cell above.


In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
labuser13495410_1768540662,pii_data


## B. Setting up PII data


### B1. Explore the customers_silver table

In the classroom setup above, we created a table named **customers_silver** within the **pii_data** schema in Unity Catalog. This table contains PII data, such as individuals names, addresses, and loyalty scores.

1. Let's review the customer data in the next cell using the default catalog and schema by specifying just the table name.

In [0]:
SELECT * 
FROM customers_silver
LIMIT 10;

customer_id,tax_id,tax_code,customer_name,state,city,postcode,street,number,unit,region,district,lon,lat,ship_to_address,valid_from,valid_to,units_purchased,loyalty_segment
11123757,null,null,"SMITH, SHIRLEY",IN,BREMEN,46506.0,N CENTER ST,521.0,null,Indiana,50.0,-86.1465825,41.4507625,"IN, 46506.0, N CENTER ST, 521.0",1532824233,1548137353,34,3
30585978,null,null,"STEPHENS, GERALDINE M",OR,ADDRESS,0,NO SITUS,null,null,null,null,-122.1055158,45.374317,"OR, 0, NO SITUS, nan",1523100473,null,18,3
349822,null,null,"GUZMAN, CARMEN",VA,VIENNA,22181,HILL RD,2860,null,VA,null,-77.2941261,38.88303270000001,"VA, 22181, HILL RD, 2860",1522922493,null,5,0
27652636,null,null,"HASSETT, PATRICK J",WI,VILLAGE OF NASHOTAH,53058.0,IVY LANE,W333N 5591,null,null,null,-88.40951700000002,43.1213789,"WI, 53058.0, IVY LANE, W333N 5591",1531834357,1558052195,7,1
14437343,null,null,"HENTZ, DIANA L",OH,COLUMBUS,43228.0,ALLIANCE WAY,5706,null,OH,FRA,-83.158438,39.97821810000001,"OH, 43228.0, ALLIANCE WAY, 5706",1517227530,null,0,0
20441596,null,null,"TIRADO, MARCO A",NY,Otselic,13072,County Road 16,2792,null,NY,Chenango,-75.7505808,42.7172722,"NY, 13072, County Road 16, 2792",1519335250,null,24,3
5945686,null,null,"SKORA, BRIAN S",MI,null,48205.0,E 8 MILE RD,16414.0,null,null,null,-82.950874,42.4499233,"MI, 48205.0, E 8 MILE RD, 16414.0",1518988242,null,7,1
5385771,null,null,"SLAWEK, DEAN J",PA,null,19147-3204,FITZWATER ST,328,null,null,null,-75.14920550000002,39.9389473,"PA, 19147-3204, FITZWATER ST, 328",1518239268,null,18,3
1427940,null,null,"REAVES, LIONEL C",VA,HOT SPRINGS,24445.0,HOT SPRINGS RD,6419.0,null,null,null,-79.90497859999998,37.8949737,"VA, 24445.0, HOT SPRINGS RD, 6419.0",1529087690,null,10,2
10457387,null,null,"BONGIOVANNI, KELLY M",IN,VINCENNES,47591,JERRY ST,2006.0,null,Indiana,42.0,-87.519002,38.662178,"IN, 47591, JERRY ST, 2006.0",1535887733,null,9,2


### B2. Create the View customers_gold_view

1. Let's create a view named **customers_gold_view** that presents a processed view of the **customers_silver** table data by averaging the units purchased by each customer.


In [0]:
CREATE OR REPLACE VIEW customers_gold_view AS
SELECT 
  customer_id, 
  state, 
  avg(units_purchased) as average_units_purchased, 
  loyalty_segment
FROM customers_silver
GROUP BY customer_id, state, loyalty_segment;

2. Let's display the view **customers_gold_view**. Confirm it contains aggregated information for each **customer_id**.

In [0]:
SELECT * 
FROM customers_gold_view
ORDER BY customer_id;

customer_id,state,average_units_purchased,loyalty_segment
1668,VT,5.0,0
2405,VT,3.0,0
2489,UT,3.0,0
2566,SD,7.0,1
4121,UT,3.0,0
4205,VT,12.0,3
4325,SD,3.0,0
4425,SD,2.0,0
5937,UT,9.0,2
6847,VT,8.0,2


### B3. View the Catalog Explorer

Catalog Explorer is a user interface tool that allows users to browse, explore, and manage data assets such as schemas, tables, models, and functions within a data catalog. It provides functionality for data discovery, including viewing schema details, previewing sample data, and exploring entity relationships, as well as management capabilities for catalogs, permissions, and data sharing

1. Run the below cell to print out the name of your catalog in Unity Catalog.

In [0]:
%python
print(f"Your Catalog Name: {DA.catalog_name}")

Your Catalog Name: labuser13495410_1768540662



To open the Catalog Explorer, complete the following:

1. Click the **Catalog** icon in the sidebar directly to the left of this notebook. It is the third icon from the top.

2. Find and expand your catalog name using the information from above.

3. Expand the schema **pii_data**. This will display a list of available objects, such as tables, views, volumes, and functions.

4. Expand the **Tables** option if it's available; otherwise, you'll directly see the available tables.

5. Select the schema **pii_data**, right-click on it, and select *Open in Catalog Explorer* to see detailed information regarding the schema. This will open in a new tab.

6. In the Catalog Explorer, you will see the available objects in the schema. Select **customers_gold_view**.

7. In the view, you can see tabs such as **Sample Data**, **Details**, and **Permissions** (which we'll cover shortly).

8. Leave the Catalog Explorer tab open and return back to this notebook.


<br></br>
**Example**

![View Catalog Explorer](./Includes/images/view_catalog_explorer.png)

## C. Controlling access to data

In this section we're going to configure permissions on data objects we created. To keep things simple, we will show you how to grant privileges to everyone. 

If you're working with a group, you can have others in the group test your work by attempting to access your data objects.

### C1. Three Level Namespace Query for Your View

The output of the following cell represents a query you could have others in your group run to attempt to access your **customers_gold_view**, using the three-level namespace: **Catalog.Schema.View**.

1. Run the code below to view the dynamically set `DA.catalog_name` course variable value. Remember, the `DA` object is a Databricks Academy variable set dynamically during the classroom setup scripts for the labs.

In [0]:
SELECT DA.catalog_name;

catalog_name
labuser13495410_1768540662


2. Run the query below using just the view name **customers_gold_view**. Notice that this works as well. 

    This is because the classroom setup script sets the default catalog to your catalog, and the default schema to **pii_data** and enables you to simply reference the table name without the catalog and schema.

In [0]:
SELECT *
FROM customers_gold_view

customer_id,state,average_units_purchased,loyalty_segment
11123757,IN,34.0,3
30585978,OR,18.0,3
349822,VA,5.0,0
27652636,WI,7.0,1
14437343,OH,0.0,0
20441596,NY,24.0,3
5945686,MI,7.0,1
5385771,PA,18.0,3
1427940,VA,10.0,2
10457387,IN,9.0,2



#### C1.1 View Privileges Notes
If someone else were to run this query, this would currently fail since no privileges have been granted yet. Only you (the owner) can access the view at the current time. **By default, only data owners (and admins) can see the data objects that were just created**. 

In order to access any data objects, users need appropriate permissions for the data object in question (a view, in this case), as well as all containing elements (the schema and catalog).

Unity catalog's security model accommodates two distinct patterns for managing data access permissions:

1. Granting permissions en masse by taking advantage of Unity Catalog's privilege inheritance.

1. Explicitly granting permissions to specific objects. This pattern is quite secure, but involves more work to set up and administer.

We'll explore both approaches to provide an understanding of how each one works.

### C2. Inherited privileges

As we've seen, securable objects in Unity Catalog are hierarchical and follow a Three Level Namespace. Privileges are inherited downward and using this feature makes it easy to set up default access rules for your data. 

Using privilege inheritance, let's build a permission chain that will allow anyone to access the view **customers_gold_view** and other objects in the same catalog and schema.
<br></br>
```

  GRANT USE CATALOG ON CATALOG ${DA.catalog} TO `account users`;

  GRANT USE SCHEMA,SELECT ON CATALOG ${DA.catalog}.example TO `account users`

```
All of these permissions were granted at the catalog level with one single statement. As convenient as this is, there are some very important things to keep in mind with this approach:

* The grantee (everyone, in this case) now has the **`SELECT`** privilege on **all** applicable objects (that is, tables and views) in **all** schemas within the catalog

* This privilege will also be extended to any future tables/views, as well as any future schemas that appear within the catalog

While this can be very convenient for granting access to hundreds or thousands of tables, we must be very careful how we set this up when using privilege inheritance because it's much easier to grant permissions to the wrong things accidentally. Also keep in mind the above approach is extreme. A slightly less permissive compromise can be made, while still leveraging privilege inheritance, with the following two grants. Note, you don't need to run these statements; they're merely provided as an example to illustrate the different types of privilege structures you can create that take advantage of inheritance.

Basically, this pushes the `USE SCHEMA` and `SELECT` down a level, so that grantees only have access to all applicable objects in the newly created schema.


1. Below is an *example* query to grant permissions to do the following:

  - The ability to use/access your catalog specified by **${DA.catalog_name}**.

  - The ability to use/access any schemas within your catalog.

  - The ability to perform SELECT operations (read data) in your catalog.

  - This allows members of the *account users* group to browse and query data within the catalog and its schemas.

**NOTE: The query result will return an UNAUTHORIZED_ACCESS error. Why? You must be the catalog's owner to grant such privileges, and you are not the owner of the catalog. This shared training workspace and your catalog has been created for you by the admin. Depending on your organization's permissions, you may also encounter this scenario in your environment.**

**With the correct permissions this would enable all account users to access the specified catalog.**

In [0]:
%python
spark.sql(f'GRANT USE CATALOG, USE SCHEMA, SELECT ON CATALOG {DA.catalog_name} TO `account users`')

---------------------------------------------------------------------------
UnknownException                          Traceback (most recent call last)
File <command-5421378864750829>, line 1
----> 1 spark.sql(f'GRANT USE CATALOG, USE SCHEMA, SELECT ON CATALOG {DA.catalog_name} TO `account users`')

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:875, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    872         _views.append(SubqueryAlias(df._plan, name))
    874 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 875 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    876 if "sql_command_result" in properties:
    877     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = 

2. Run the `DESCRIBE CATALOG` statement to view information about the catalog. View the results. Notice that you can view the *Owner* of a catalog. In this example, you are not the owner of the catalog and do not have the ability to grant permissions.

In [0]:
%python
r = spark.sql(f'DESCRIBE CATALOG {DA.catalog_name}')
display(r)

### C3. Check Privileges in Unity Catalog

There are two ways to check the applied permissions on objects: 

- Catalog Explorer
- Code

#### C3.1 Check Permissions in Catalog Explorer

Check privileges on a catalog using the Catalog Explorer by completing the following steps:

1. Click the **Catalog** icon in the sidebar, directly to the left of this notebook. It is the third icon from the top.

2. Find and expand your unique catalog name.

3. Right-click on your catalog name and select *Open in Catalog Explorer* to view detailed information about the catalog. This will open in a new tab.

4. In the Catalog Explorer, you will see the available schemas in the catalog.

5. In the top navigation bar, you will see **Overview**, **Details**, **Permissions**, **Policies**.

6. Select **Permissions**.

7. Notice that in **Permissions** you see that you have *ALL PRIVILEGES* on this catalog. *ALL PRIVILEGES* current provides the following:

    - **Prerequisite:** *USE CATALOG, USE SCHEMA*
    - **Metadata:** *APPLY TAG, BROWSE*
    - **Read:** *EXECUTE, READ VOLUME, SELECT*
    - **Edit:** *MODIFY, REFRESH, WRITE VOLUME*
    - **Create:** *CREATE FUNCTION, CREATE MATERIALIZED VIEW, CREATE MODEL, CREATE SCHEMA, CREATE TABLE, CREATE VOLUME*

For more information view the [Privilege types by securable object in Unity Catalog](https://docs.databricks.com/en/data-governance/unity-catalog/manage-privileges/privileges.html) documentation.

**Example Catalog Explorer Image**

![Unity Catalog Privileges](./Includes/images/uc_permissions.png)

#### C3.2 Check Permissions with Code
1. You can also use the [`SHOW GRANTS command`](https://docs.databricks.com/en/sql/language-manual/security-show-grant.html) to check the grants for your catalog. This command is a versatile SQL statement used to display grants about various database objects. In this case, we'll apply it to a catalog following the next syntax: 
```

SHOW GRANTS [ principal ] ON securable_object


```

Execute the cell below and view the results.


In [0]:
%python
r = spark.sql(f'SHOW GRANTS ON CATALOG {DA.catalog_name}')
display(r)

### C4. Grant Explicit privileges on Schema or Objects


1. Show privileges on the **pii_data** schema within your catalog. Notice that only you have access to that schema.

In [0]:
SHOW GRANTS ON SCHEMA pii_data;

2. Show privileges on the **customers_gold_view** view within your catalog's **pii_data** schema. Notice that only you have access to that view.

In [0]:
SHOW GRANTS ON VIEW customers_gold_view;

3. Using explicit privilege grants on Schema level, let's build a permission chain that will allow anyone to access the **customers_gold_view** view.

**NOTE:** This code will execute successfully because you created the schema and view within the catalog. However, you will need to provide access to the catalog for other users to access the objects within it. In this lab environment, we have restricted you from granting privileges on your catalog.

In [0]:
%python
## This permission grants users the ability to access and interact with the specified catalog, allowing them to see and query schemas within it.
## Granting privileges on your catalog will not work in this lab - commented out
##
## spark.sql(f'GRANT USE CATALOG ON CATALOG {DA.catalog_name} TO `account users`')
##

## This permission grants the account users group the ability to access and interact with the pii_data schema in the specified catalog.
spark.sql(f'GRANT USE SCHEMA ON SCHEMA {DA.catalog_name}.pii_data TO `account users`')

## This permission grants the account users group the ability to query (select data from) the customers_gold_view view in the pii_data schema of the specified catalog.
spark.sql(f'GRANT SELECT ON VIEW {DA.catalog_name}.pii_data.customers_gold_view TO `account users`')

With these grants in place (if you had permission to grant all access), if anyone else were to query the view again, the query still succeeds because all the appropriate permissions are in place; we've just taken a very different approach to establishing them.

This seems more complicated. One statement from earlier has been replaced with three, and this only provides access to a single view. 

Following this pattern, we'd have to do an additional `SELECT` grant for each additional table or view we wanted to permit. But this complication comes with the benefit of security. Now, user can only read the *gold* view, but nothing else. There's no chance they could accidentally get access to some other object. So this is very explicit and secure, but one can imagine it would be very cumbersome when dealing with lots of tables and views.

### C5. Check Explicit privileges


#### C5.1 Show Schema Privileges

1. Similarly to previous steps, we'll leverage the `SHOW GRANTS` statement again to check the explicit privileges granted to your schema **pii_data**.

    Notice that in the results `account users` have been added to the **pii_data** schema.

In [0]:
SHOW GRANTS ON SCHEMA pii_data;

#### C5.2 Show View Privileges

1. View the privileges on the **customers_gold_view**. Notice that in the results, `account users` have been granted access to the view **customers_gold_view**.


In [0]:
SHOW GRANTS ON VIEW pii_data.customers_gold_view;

2. View the grants on your catalog. Notice that `account users` does not have access to grant access to the catalog.

In [0]:
%python
r = spark.sql(f'SHOW GRANTS ON CATALOG {DA.catalog_name}')
display(r)

### C6. Revoking Privileges

No data governance platform would be complete without the ability to revoke previously issued grants. In preparation for testing the next approach to granting privileges, let's unwind what we just did using `REVOKE`.

1. Revoke the usage of `account users` on your schema.

In [0]:
REVOKE USAGE ON SCHEMA pii_data FROM `account users`;
REVOKE SELECT ON VIEW pii_data.customers_gold_view FROM `account users`;

2. After revoking the access to our schema, you check the permissions again by running the query below.



In [0]:
SHOW GRANTS ON SCHEMA pii_data;

In [0]:
SHOW GRANTS ON VIEW pii_data.customers_gold_view;

### C7. Views versus tables

We've explored two different approaches to managing permissions, and we have now shown how permissions can configured such that anyone can access the **customers_gold_view** view, which processes and displays data from the **customers_silver** table. 

But suppose someone else were to try to directly access the **customers_silver** table. This could be accomplished by replacing **customers_gold_view** in the previous query with **customers_silver**.

With explicit privileges in place, the query would fail. How then, does the query against the **customers_gold_view** view work? Because the view's **owner** has appropriate privileges on the **customers_silver** table (through ownership). This property gives rise to interesting applications of views in table security, which we cover in the next section.

## D. Protecting columns and rows

Databricks provides several options for protecting columns and rows, in this section we'll use the **customers_silver** table as a source for creating **dynamic views** and applying **row filtering and column masks** to protect sensitive data:
- Dynamic View: **customers_gold_dynamic_view**
- Table: **customers_silver_with_row_filter_and_column_masks** with `ROW_FILTER` and `MASK COLUMN`

### D1. Preview the customers_silver table

1. Run the query and view the **customers_silver** table.

    Notice that the **customers_silver** table contains detailed customer information, including various attributes such as customer ID, name, contact details, and loyalty segment.

In [0]:
SELECT *
FROM customers_silver
LIMIT 10;

2. We'll use the **loyalty_segment** column to filter rows in our dynamic view and table with ROW_FILTER. 

    Run the query below to view distinct values in the **loyalty_segment** column. Notice that currently, four values are shown in the cell below.

In [0]:
SELECT DISTINCT(loyalty_segment)
FROM customers_silver
ORDER BY loyalty_segment DESC;

3. Count the number of rows in the **customers_silver** table. Confirm the table contains *28,813* rows.

In [0]:
SELECT count(*) as TotalRows
FROM customers_silver;

### D2. Dynamic Views

We have seen that Unity Catalog's treatment of views provides the ability for views to protect access to tables; users can be granted access to views that manipulate, transform, or obscure data from a source table, without needing to provide direct access to the source table.

Dynamic views provide the ability to do fine-grained access control of columns and rows within a table, conditional on the principal running the query. Dynamic views are an extension to standard views that allow us to do things like:
* partially obscure column values or redact them entirely
* omit rows based on specific criteria

Access control with dynamic views is achieved through the use of functions within the definition of the view. These functions include:
* `current_user()`: returns the email address of the user querying the view
* `is_account_group_member()`: returns TRUE if the user querying the view is a member of the specified group
* `is_member()`: returns TRUE if the user querying the view is a member of the specified workspace-local group

**NOTE:** Databricks generally advises against using the `is_member()` function in production, since it references workspace-local groups and hence introduces a workspace dependency into a metastore that potentially spans multiple workspaces.

1. Run the cell below to use the `is_account_group_member()` function to check if you are part of the `supervisors` group.

    Notice that it returns the value *false*, indicating that you are not part of the `supervisors` group.

In [0]:
SELECT is_account_group_member('supervisors')

### D3. Redacting 'Customer ID' and hiding 'Loyalty > 2' customers

Suppose we want everyone to be able to see aggregated data trends from the **customers_silver** table, but we don't want to disclose customer PII to everyone. 

Let's create a view to redact the **customer_id** column and hide high loyalty customers, so that only members of `supervisors` group can see it using the `is_account_group_member()` function.

Column redactions are performed using `CASE` statements and row filtering is done by applying the conditional as a `WHERE` clause.

This view performs the following:
- This view dynamically redacts **customer_id** based on the user's group membership.
- `Supervisors` see the actual customer_id, others see 9999999.
- Non-supervisors only see customers with a **loyalty_segment** less than *3*.
- The view calculates the average units purchased and includes **state** and **loyalty_segment**.

In [0]:
CREATE OR REPLACE VIEW customers_gold_dynamic_view AS
SELECT 
  CASE WHEN        -- Redact customer_id column if user is not a supervisor
    is_account_group_member('supervisors') THEN customer_id 
    ELSE 9999999
  END AS customer_id,
  state, 
  avg(units_purchased) as average_units_purchased, 
  loyalty_segment
FROM customers_silver
WHERE
  CASE WHEN          -- Redact rows where loyalty_segment 3 or above if the user is not a supervisor
    is_account_group_member('supervisors') THEN TRUE  -- When true, return all rows
    ELSE loyalty_segment < 3                          -- When false, return rows less than 3
  END
GROUP BY customer_id, state, loyalty_segment
ORDER BY customer_id;

### D4. Dynamic View Results

1. Now, let's query the view. Notice the **customer_id** column is now *9999999* and displaying only rows with **loyalty_segment** below 3

In [0]:
SELECT * 
FROM customers_gold_dynamic_view
ORDER BY loyalty_segment DESC;

2. Count the number of rows in the **customers_gold_dynamic_view** dynamic view. Confirm the table contains 19,176 rows (original table had 28,813), filtering out rows that you are not allowed to view because you are not part of the `supervisors` group.

In [0]:
SELECT count(*) as TotalRows
FROM customers_gold_dynamic_view;

**Does this output surprise you?**

As the owner of the view and table, you do not need any privileges to access these objects, yet when querying the view, we see redacted columns. This is because of the way the view is defined. As a regular user (one who is not a member of the `supervisors` group), the **customer_id** column is redacted and the high loyalty customer records are hidden.

## E. Row Filters and Column Masks (Introduced in 2024) 

This newly introduced feature enables data owners to mask columns and hide rows in similar ways to Dynamic Views - without having to create another data object.

- **Row filters** allow you to apply a filter to a table so that queries return only rows that meet the filter criteria. You implement a row filter as a SQL user-defined function (UDF). Python and Scala UDFs are also supported, but only when they are wrapped in a SQL UDF.

- **Column masks** let you apply a masking function to a table column. The masking function gets evaluated at query runtime, substituting each reference of the target column with the results of the masking function. For most use cases, column masks determine whether to return the original column value or redact it based on the identity of the invoking user. Column masks are expressions written as SQL UDFs or as Python or Scala UDFs that are wrapped in a SQL UDF.

### E1. Creating a Row Filter

1. To create a row filter, you need to write a UDF to define the filter policy and then apply it to a table with an ALTER TABLE statement. Alternatively, you can specify a row filter for a table in the initial CREATE TABLE statement. Each table can have only one row filter. A row filter accepts zero or more input parameters where each input parameter binds to one column of the corresponding table.

    Our UDF will leverage `is_account_group_member()` function to evaluate if the current user is a member of the 'supervisors' account group. Run the code and view the output. Again, you are not part of the group.

In [0]:
SELECT is_account_group_member('supervisors')

2. Let's create the UDF `loyalty_row_filter` using the `is_account_group_member` function to drive the behavior for row filtering: if the user is not a member of the group, it will filter the **loyalty_segment** column for values less than 3.

In [0]:
DROP FUNCTION IF EXISTS loyalty_row_filter;

CREATE OR REPLACE FUNCTION loyalty_row_filter(loyalty_segment STRING)
RETURNS BOOLEAN
RETURN IF(is_account_group_member('supervisors'), true, loyalty_segment < 3);

3. Now lets create the table **customers_silver_with_row_filter_and_column_masks** from the **customers_silver** table and view it's results. 

    Notice since the `ROW_FILTER` from above is not applied yet, the query below will retrieve and include all rows for a total of 28,670 rows.

In [0]:
-- Drop the table if it exists for demo purposes
DROP TABLE IF EXISTS customers_silver_with_row_filter_and_column_masks;

-- Create a new table and apply the row filter
CREATE OR REPLACE TABLE customers_silver_with_row_filter_and_column_masks AS 
SELECT 
  customer_id,
  state, 
  avg(units_purchased) as average_units_purchased, 
  loyalty_segment
FROM customers_silver
GROUP BY customer_id, state, loyalty_segment
ORDER BY customer_id;


-- View the new table
SELECT count(*) AS TotalRows
FROM customers_silver_with_row_filter_and_column_masks;

4. Now let's assign the `loyalty_row_filter` function with `WITH ROW FILTER` using an `ALTER TABLE` statement on the table `customers_silver_with_row_filter_and_column_masks` to the **loyalty_segment** column.

    [ROW FILTER clause](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-row-filter.html#row-filter-clause) documentation.

In [0]:
ALTER TABLE customers_silver_with_row_filter_and_column_masks 
SET ROW FILTER loyalty_row_filter ON (loyalty_segment);

5. Let's confirm the `ROW FILTER` has been applied to the table with a [`DESCRIBE EXTENDED`](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-aux-describe-table.html) statement.

    Run the query and view the results. Scroll to the bottom of the table to the **Row Filter** row in the **col_name** column. Notice that a row filter has been applied.

In [0]:
DESCRIBE EXTENDED customers_silver_with_row_filter_and_column_masks;

6. Let's confirm that the `ROW_FILTER` is working by applying a `DISTINCT` on the **loyalty_segment** column in the table. The results should be between *0* and *2*.

In [0]:
SELECT distinct(loyalty_segment)
FROM customers_silver_with_row_filter_and_column_masks
ORDER BY loyalty_segment DESC;

7. Let's count the total number of rows in the **customers_silver_with_row_filter_and_column_masks** table. Notice that the table now contains *19,176* rows instead of the original *28,670*, with rows filtered out because you are not part of the assigned `supervisors` group.


In [0]:
SELECT count(*) AS TotalRows
FROM customers_silver_with_row_filter_and_column_masks;

### E2. Create a Column Mask

To apply column masks, create a UDF and apply it to a table column using an `ALTER TABLE` statement.

1. Let's create the function `redact_customer_id` and use the `is_account_group_member` function to control its behavior. If `is_account_group_member` returns `true`, it returns the actual **customer_id**. Otherwise, it returns the value *9999999*.


In [0]:
DROP FUNCTION IF EXISTS redact_customer_id;

CREATE OR REPLACE FUNCTION redact_customer_id(customer_id BIGINT)
RETURN CASE WHEN is_account_group_member('supervisors') 
  THEN customer_id 
  ELSE 9999999
END;

2. Now, let's assign the `redacted_customer_id` function using `SET MASK` in an `ALTER TABLE` statement for the table **customers_silver_with_row_filter_and_column_masks**.

    [Column mask clause](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-column-mask.html#column-mask-clause) documentation.

    [ALTER COLUMN clause](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-alter-table-manage-column.html#alter-column-clause) documentation.


In [0]:
ALTER TABLE customers_silver_with_row_filter_and_column_masks
  ALTER COLUMN customer_id 
  SET MASK redact_customer_id;

3. Let's confirm that the `Column Mask` has been applied to the table using a [`DESCRIBE EXTENDED`](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-aux-describe-table.html).

    Run the query and view the results. Scroll to the bottom of the output to the **Column Mask** row in the **col_name** column. You should notice that a column mask has been applied.


In [0]:
DESCRIBE EXTENDED customers_silver_with_row_filter_and_column_masks

### E3. Table with Row Filtering and Column Mask Results

1. Now, let's query the table. You should notice that the column masking in **customer_id** shows the value *9999999*, and the row filtering displays only rows where the **loyalty_segment** is below 3.


In [0]:
SELECT * 
FROM customers_silver_with_row_filter_and_column_masks
ORDER BY loyalty_segment DESC
LIMIT 20;

## F. Compare Results of Dynamic Views with Row Filters and Column Masks

The query below provides a quick row count comparison between the two tables we've created, with matching values, to demonstrate that both can achieve the same results: **customers_gold_dynamic_view** and **customers_silver_with_row_filter_and_column_masks**.

1. Run the query and view the results. You should notice that the view and table return the same row count.

In [0]:
SELECT 
  (SELECT count(*) FROM customers_silver_with_row_filter_and_column_masks) = 
  (SELECT count(*) FROM customers_gold_dynamic_view) AS equal_row_count

### F1. Dynamic Views vs Row Filters - Discuss

- Dynamic views, row filters, and column masks all let you apply complex logic to tables and process their filtering decisions at query runtime.

- Use dynamic views if you need to apply transformation logic such as filters and masks to read-only tables, and if it is acceptable for users to refer to the dynamic views using different names. 

- Use row filters and column masks if you want to filter or compute expressions over specific data but still provide users access to the tables using their original names.




## G. Tagging 
Tags are attributes with keys and optional values that can be applied to securable objects in Unity Catalog to organize and categorize them.

- Supported objects for tagging include catalogs, schemas, tables, columns, volumes, views, registered models, and model versions.

- Tags simplify search and discovery of tables and views using workspace search functionality.

- You can assign up to 20 tags per object, with key length up to 255 characters and value length up to 1000 characters.

- Tags can be added and managed through Catalog Explorer UI or SQL commands (for Databricks Runtime 13.3+).

- Tags can be used for data classification, security, lifecycle management, compliance, and project management.

### G.1 Set Tags to Table and Column
Let's set a tag to the **customers_silver** table, this can be achieved using an `ALTER TABLE` statement or via the UI in Catalog Explorer. 

**NOTE:** Remember this can be set to other objects stated above (catalogs, schemas, columns, etc):

### G2. Tagging via ALTER TABLE

In [0]:
-- TABLE TAGS
ALTER TABLE customers_silver 
SET TAGS (
  'quality'='silver',
  'domain'='customer'
  );


-- COLUMN TAGS
ALTER TABLE customers_silver 
  ALTER COLUMN customer_id SET TAGS ("compliance" = "GDPR");

### G3. Tagging via Catalog Explorer

1. Using the Catalog Explorer, navigate to the **customers_silver** table in your catalog within the **pii_schema**. 

    In the **Overview** tab, you will find the "Tagging" section on the right side of the panel. Notice the tags we defined in the previous cell

**Example Tag in Catalog Explorer**

![Tagging in Catalog Explorer](./Includes/images/uc_tag.png)

## H. Discoverability

Unity Catalog offers robust data discovery capabilities, allowing users to easily search for and locate data assets across their organization. The platform provides a structured way to tag, document, and manage metadata, enabling comprehensive search functionality that utilizes lineage information and enforces security based on user permissions

There are two ways to leverage tags for discoverability:


### H1. Search Bar

1. Using the syntax such as `tag:value`. In our example should be `domain:customer`. The more tags added, the finer the results.

    Run the query below and copy the values and paste them into the search bar on top and press enter to see the results. 



**Example Results**

![Discoverability](./Includes/images/search_bar.png)

**NOTE:** Please take into consideration the image may have a different catalog name from yours.

In [0]:
SELECT concat('catalog:',DA.catalog_name,' ','domain:customer') as use_in_search_bar


### H2. Queries

Alternatively, run the query below leveraging the `INFORMATION_SCHEMA.TABLE_TAGS` filtering with the `customers_silver` table.
Be aware of the tables below to retrieve tags from the different objects:

- `INFORMATION_SCHEMA.CATALOG_TAGS`
- `INFORMATION_SCHEMA.SCHEMA_TAGS`
- `INFORMATION_SCHEMA.TABLE_TAGS`
- `INFORMATION_SCHEMA.COLUMN_TAGS`
- `INFORMATION_SCHEMA.VOLUME_TAGS`


In [0]:
SELECT * 
FROM INFORMATION_SCHEMA.TABLE_TAGS
WHERE TABLE_NAME = 'customers_silver'

## I. Lineage

1. Data lineage is a key pillar of any data governance solution. In the **Lineage** tab, we can identify elements that are related to the selected object:
* With **Upstream** selected, we see objects that gave rise to this object, or that this object uses. This is useful for tracing the source of your data.
* With **Downstream** selected, we see objects that are using this object. This is useful for performing impact analyses.
* The lineage graph provides a visualization of the lineage relationships.

You can access the lineage of a table in Catalog Explorer by selecting your table, in this case **customers_silver**, in the _"lineage"_ tab there is a button _"see lineage graph"_ to display the results shown below. 

**Example**

**Note:** Take into consideration some information such as catalog won't match your view.

![Lineage](./Includes/images/lineage.png)

## J. AI generated Documentation

AI-generated documentation for Unity Catalog enables automatically generate descriptions for tables and columns. The feature uses a custom-built large language model (LLM) to generate metadata based on table schemas and column names.

- Available for catalogs, schemas, tables, columns, functions, models, and volumes.
- Saves time and reduces manual effort in documenting data assets.
- Improves search functionality within Databricks workspaces.
- Users need appropriate permissions (object owner or MODIFY privilege) to view, edit, and save AI-generated comments.


### J1. Tables
1. We'll auto-generate concise and informative table and column comments for Unity Catalog, leveraging DatabricksIQ. 

    In the Catalog Explorer search for the "customers_silver" table and in the "Overview" Tab, DatabricksIQ will suggest a "_AI Suggested Description_", you can edit and adjust it as per your needs or accept such a recommendation. Also is possible to update the description later if needed.

**Example**

![AI Generated Table Description](./Includes/images/table_ai_desc.png)


### J2. Columns

1. In the same manner with the table description, under the "Overview" tab, below where the schema is presented. There is a button "AI Generate", after clicked, DatabricksIQ will generate a description for each column as shown in the image below, you can also revert or adjust as needed.

**Example**


![AI Generated Column Description](./Includes/images/table_column_ai_desc.png)

## K. Insights
You can use the Insights tab in Catalog Explorer to view the most frequent recent queries and users of any table registered in Unity Catalog. The Insights tab reports on frequent queries and user access for the past 30 days.

You must have the following permissions to view frequent queries and user data on the Insights tab.
* SELECT privilege on the table.
* USE SCHEMA privilege on the table’s parent schema.
* USE CATALOG privilege on the table’s parent catalog.

Metastore admins have these privileges by default.

In the Insights tab for a table, you can view 
1. Frequently used queries and notebooks
1. Frequently used dashboards
1. Frequent Users
1. Other tables frequently joined with the table in question

The Insights tab can also help identify tables that are no longer used by applications. These tables can then be tagged for future cleanup.


&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>